<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#1.-Setting-Parameter" data-toc-modified-id="1.-Setting-Parameter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Setting Parameter</a></span></li><li><span><a href="#2.-Data-Collect" data-toc-modified-id="2.-Data-Collect-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Data Collect</a></span></li><li><span><a href="#3.-Example-Input-Data" data-toc-modified-id="3.-Example-Input-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Example Input Data</a></span></li><li><span><a href="#4.-Example-Save-Data" data-toc-modified-id="4.-Example-Save-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Example Save Data</a></span></li></ul></div>

##### CSV 타입의 BasicData를 업로드하는 Test Code

----------
- CSV 형태의 데이터파일 업로드 Test Code
- CSV Data Upload 방식에는 4가지 타입이 존재
    1. **Basic : 원 데이터를 변형 없이 그대로 업로드하거나 컬럼을 삭제 및 이름 변경 후 업로드하는 방법**
    2. Selected Data : 원 데이터에서 특정 컬럼이 특정 조건을 만족하는 데이터만을 선택하여 업로드하는 방식 (1번 타입의 방법 기능을 포함)
        - 특정 조건 종류
            - Equal/Above/Below/Exceeded/Less than/Exception
    3. TimeDuplication Data : 시간정보가 중복되어 정보가 수집된 데이터로 중복된 시간을 처리하여 업로드하는 방법 (1,2번 타입의 방법 기능을 포함)
        - 중복 처리 방법
            -Remove/Sum/Average/Max/Min
    4. Folder Data (highCapacity) : 하나의 폴더안에 여러개의 데이터가 저장되어 있으며 폴더로 데이터를 업로드하는 방법 (대용량 업로드/1,2번 타입의 기능을 포함)

# Import

In [1]:
import pandas as pd
import os
import sys

sys.path.append("../")
sys.path.append("../..")

from Clust.setting.influx_setting_KETI import CLUSTDataServer2 as ins
from clust.ingestion.influx import influx_Client_v2 as iC

import clust.ingestion.CSVtoInflux.csvCollector as cC

# 1. Setting Parameter


- parameter 는 KETIPreDataCollection.collectorParameter 의 parameter 로 호출 가능
```
###예시###
import KETIPreDataCollection.collectorParameter as cP
cP.parameter
```

In [2]:
####### db connection parameter
db_client = iC.influxClient(ins)

In [3]:
####### Folder / File 업로드 방식 고르시
upload_type_list = ["File", "Folder"]
upload_type = upload_type_list[0]

In [4]:
####### read&clean type 고르기 parameter
data_read_type_list = ["basic", "selectedData", "highCapacity"]
data_read_type = data_read_type_list[0]

In [5]:
####### 기본 Data 관련 저장에 필요한 정보 기입
db_name = "test_data_type1"
ms_name = "test1"
path = "./exampleData/Type1_BasicData.csv"
encoding = "utf-8"

In [6]:
####### 저장될 데이터의 시간 스템프로 지정될 column 을 기입하는 parameter
time_column = "자치구 기준일"

In [7]:
####### clean type 에 따라 data read type의 parameter 가 달라짐
selected_datas = None

In [8]:
####### 저장하는 column 선택하는 parameter
selected_columns = [{"Selected_columns":['종로구 전체', '종로구 추가']},
                    {"Rename_columns":['jongno1', 'jongno2']}]

In [9]:
####### duplicated time column processing method 정하는 parameter
dtcpm = None

In [ ]:
####### 저장 데이터의 field type 지정하는 parameter
field_type = "float"

In [10]:
####### 최종 Parameter
parameter = {
    "uploadType" : upload_type,
    "dataReadType" : data_read_type,
    "dbName" : db_name,
    "msName" : ms_name,
    "filePath" : path,
    "selectedDatas" : selected_datas, # data read type 이 selectedData 인 경우 필요한 param
    "timeColumnName" : time_column,
    "selectedColumns" : selected_columns,
    "duplicatedTimeColumnProcessingMethod" : dtcpm,
    "encoding" : encoding,
    "fieldType" : field_type # int / string
}
print("parameter success")

parameter success


# 2. Data Collect

In [12]:
csv_collector = cC.Collector(parameter, db_client)
csv_collector.collect()

===========data clean success===========
Writing Data ...
=========== ms name : test1 ===========
                     jongno1  jongno2
time                                 
2020-02-29 10:00:00     11.0      0.0
2020-02-28 10:00:00     11.0      0.0
========== create bucket ==========
========== write success ==========


# 3. Example Input Data

In [17]:
ex_data = pd.read_csv(path)

In [18]:
ex_data.head()

,자치구 기준일,종로구 전체,종로구 추가,중구 전체,중구 추가,용산구 전체,용산구 추가,성동구 전체,성동구 추가,광진구 전체,...,서추구 추가,강남구 전체,강남구 추가,송파구 전체,송파구 추가,강동구 전체,강동구 추가,기타 전체,기타 추가,수집일
0,2021.05.21.00,737.0,0.0,646.0,1.0,1092.0,5.0,1142.0,4.0,1363.0,...,3.0,2320.0,12.0,2405.0,20.0,1624.0,10.0,4057.0,8.0,11:39.0
1,2021.05.20.00,737.0,3.0,645.0,4.0,1087.0,3.0,1138.0,5.0,1354.0,...,6.0,2308.0,25.0,2384.0,24.0,1614.0,13.0,4049.0,5.0,11:40.0
2,2021.05.19.00,734.0,6.0,641.0,5.0,1084.0,3.0,1133.0,5.0,1347.0,...,9.0,2283.0,32.0,2360.0,17.0,1601.0,13.0,4044.0,12.0,11:39.0
3,2021.05.18.00,728.0,4.0,637.0,2.0,1081.0,4.0,1128.0,2.0,1342.0,...,6.0,2251.0,15.0,2343.0,12.0,1588.0,12.0,4030.0,9.0,11:39.0
4,2021.05.17.00,724.0,2.0,635.0,1.0,1077.0,2.0,1126.0,6.0,1333.0,...,3.0,2235.0,13.0,2332.0,12.0,1576.0,17.0,4021.0,2.0,11:39.0


# 4. Example Save Data 

In [15]:
db_client = iC.influxClient(ins)
save_data = db_client.get_data(db_name, ms_name)

In [16]:
save_data.head()

,jongno1,jongno2
time,,
2020-02-28 10:00:00+00:00,11.0,0.0
2020-02-29 10:00:00+00:00,11.0,0.0
2020-03-01 10:00:00+00:00,11.0,0.0
2020-03-02 10:00:00+00:00,11.0,0.0
2020-03-03 10:00:00+00:00,11.0,0.0
